In [83]:
from folpy.utils.parser.parser import Parser
from folpy.semantics.classes import Quasivariety
from functools import reduce

Importamos el lattice N5 y la cadena de 2 elementos. Luego generamos la cuasivariedad generada por N5 y obtenemos el lattice que vamos a estudiar, en este caso una subálgebra del producto N5*C2.

In [2]:
N5 = Parser("models/N5.model").parse()
C2 = Parser("models/2chain.model").parse()

Q = Quasivariety([N5])
A_inicial = list((C2*N5).substructures())[2].continous()[0]

print("Cantidad de SRI: %s (salvo isomorfismos)" % len(Q.generate_rsi())) 

Cantidad de SRI: 2 (salvo isomorfismos)


Generamos la descomposición subdirecta del lattice, y nos quedamos con el lattice isomorfo pero que es un producto subdirecto:

In [10]:
from utils import descomp_subdirecta
homeo = descomp_subdirecta(A_inicial, Q)

P = homeo.target
A = P.restrict(list(homeo.image()))

print("Cantidad de factores: %s" % len(P.indexes()))

for i in P.indexes():
    print("Cantidad de elementos del álgebra %s: %s" % (i,len(P.factors[i])) )

Cantidad de factores: 4
Cantidad de elementos del álgebra 0: 2
Cantidad de elementos del álgebra 1: 5
Cantidad de elementos del álgebra 2: 2
Cantidad de elementos del álgebra 3: 2


A partir de los factores de la descomposición, generamos las congruencias que dan esa descomposición, que van a ser las CMI del reticulado de congruencias de $A$.

In [12]:
deltas = {}
for i in P.indexes():
    deltas[i] = P.projection(i).composition(homeo).kernel()
    print("delta_%s: %s" % (i, deltas[i]))

delta_0: Congruence([|0, 1, 2, 3, 4, 5|, |6, 7|])
delta_1: Congruence([|0, 3|, |1, 4|, |2, 5|, |6|, |7|])
delta_2: Congruence([|0, 1, 2|, |3, 4, 5, 6, 7|])
delta_3: Congruence([|0, 3, 6|, |1, 2, 4, 5, 7|])


Cuando analizamos la congruencia, vemos que las CMI de $A$ están ordenadas de la siguiente manera:

```
d0    d3
  \  /
   d1     d2
```

Sabemos que las congruencias que tomemos para el sistema van a ser generadas por intersecciones de CMIs, por lo tanto, viendo las congruencias como conjuntos de índices que se corresponden con las CMIs, estos conjuntos son crecientes. Además queremos que la unión de nuestros conjuntos de índice cubra todos los índices, por lo tanto va a haber al menos un $F_i$ que tenga a los indices $0, 1 $ y $3$; y otro $F_j$ que tenga a $2$.

Por lo tanto comenzamos tomando nuestro conjunto de congruencias $\{\delta_1, \delta_2\}$; es decir, los conjuntos de indices $F_1 = \{0,1,3\}$ y $F_2 = \{2\}$.

In [71]:
Fs = {}
Fs[1] = {0, 1, 3}
Fs[2] = {2}

Vamos a definir algunos operadores a partir de $A$ y $P := \prod A_i$.

 - $A' := \{ \bar{s} \in P : \ \bar{s}|_{F_i} \in A|_{F_i} \ \forall i \in I \}$
 - $\bar{A_{ij}} := \{ \bar{a} \in P : \ a_{ij} \in A_{ij} \}$

Cómo estamos en una variedad con M, se puede ver que:

 - $A = \bigcap_{i,j\in H} \bar{A_{ij}}$, donde $H := \{ ij : ij \in I^2 \}$
 - $A' = \bigcap_{i,j\in H'} \bar{A_{ij}}$, donde $H' := \{ ij \in I^2 : \exist k \in I $ tal que $ ij \in F_k \}$ 

Es claro que $H' \subseteq H$, y por lo tanto $A \subseteq A'$. Si se cumple la igualdad ($A = A'$), tenemos que todo sistema tiene solución, sino existe alguno sin solución. 


In [20]:
from utils import dos_proyeccion_extendida

H = [(i,j) for i in range(4) for j in range(4) if i < j]

A_barra = P.universe
for (i, j) in H:
    A_barra = [x for x in A_barra if x in dos_proyeccion_extendida(homeo, i, j)]

assert(sorted(A_barra) == sorted(A.universe))

In [22]:
H_prima = [(i,j) for (i,j) in H if any(i in Fs[k] and j in Fs[k] for k in Fs.keys())]

A_barra_prima = P.universe
for (i, j) in H_prima:
    A_barra_prima = [x for x in A_barra_prima if x in dos_proyeccion_extendida(homeo, i, j)]

set(A_barra_prima).difference(set(A_barra))

{(1, 3, 0, 0), (1, 4, 0, 1)}

$A'$ tiene 2 elementos más que $A$, por lo que van a existir 2 sistemas que no tengan solución para $\{\delta_1, \delta_2\}$.


La primera observación es que si los índices $1$ y $2$ están en el mísmo $F_i$ estamos consideranco el caso en donde una congruencia es $\Delta$, por lo cual no tiene mucho sentido este caso. Vamos a probar los siguientes 2 casos:

### Caso 1

 - $F_1 := \{ 0, 1, 3 \}$
 - $F_2 := \{ 0, 2 \}$

In [84]:
Fs = {}
Fs[1] = {0, 1, 3}
Fs[2] = {0, 2}
tita = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[1]])
gama = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[2]])

H_prima = [(i,j) for (i,j) in H if any(i in Fs[k] and j in Fs[k] for k in Fs.keys())]

A_barra_prima = P.universe
for (i, j) in H_prima:
    A_barra_prima = [x for x in A_barra_prima if x in dos_proyeccion_extendida(homeo, i, j)]

set(A_barra_prima).difference(set(A_barra))

set()

Aqui podemos ver que $A' = A$, por lo que todo sistema tiene solución, veamos quienes son las congruencias de este sistema.

In [85]:
tita, gama

(Congruence([|0, 3|, |1, 4|, |2, 5|, |6|, |7|]),
 Congruence([|0, 1, 2|, |3, 4, 5|, |6, 7|]))

Podemos notar que en este caso, $\gamma$ tiene 3 clases de equivalencia, por lo que ese cociente nos queda la cadena de 3 elementos. 

### Caso 2

 - $F_1 := \{ 0, 1, 3 \}$
 - $F_2 := \{ 2, 3 \}$

In [86]:
Fs = {}
Fs[1] = {0, 1, 3}
Fs[2] = {2, 3}
tita = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[1]])
gama = reduce(lambda x, y: x & y, [deltas[i] for i in Fs[2]])

H_prima = [(i,j) for (i,j) in H if any(i in Fs[k] and j in Fs[k] for k in Fs.keys())]

A_barra_prima = P.universe
for (i, j) in H_prima:
    A_barra_prima = [x for x in A_barra_prima if x in dos_proyeccion_extendida(homeo, i, j)]

set(A_barra_prima).difference(set(A_barra))

{(1, 3, 0, 0), (1, 4, 0, 1)}

In [87]:
tita, gama

(Congruence([|0, 3|, |1, 4|, |2, 5|, |6|, |7|]),
 Congruence([|0|, |1, 2|, |3, 6|, |4, 5, 7|]))

In [100]:
from folpy.semantics.lattices import LatticeQuotient

LatticeQuotient(A_inicial, gama).draw()

En este caso, $\gamma $ tiene 4 clases, y el cociente de $A/\gamma $ es el rombo, a diferencia del caso anterior. 